In [1]:

import os
import json
from zs6d_sd_dino.fused_zs6d import Fused_ZS6D
# Loading the config file:
with open(os.path.join("./zs6d_configs/bop_eval_configs/cfg_ycbv_inference_bop_sd_dino.json"), "r") as f:
    config = json.load(f)

model_type = 'dinov2_vitb14'
stride = 14
pose_estimator = Fused_ZS6D(config['templates_gt_path'], config['norm_factor_path'], model_type, stride)

2024-07-06 15:34:21,305 - ERROR - Failed to load templates or norm_factors: Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
import os
import json
import numpy as np
import torch
from zs6d_sd_dino.sd_dino.extractor_sd import load_model

"""SETUP for SD-DINO"""
SEED = 42
VER = "v1-5"
TIMESTEP = 100
img_size_sd = 960
model_type = 'dinov2_vitb14'
layer = 11 # or 9
facet = 'token' # or key token
stride = 14 # or 4

"""Reproducibility"""
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.benchmark = True


model, aug = load_model(diffusion_ver=VER, image_size=img_size_sd, num_timesteps=TIMESTEP)

/run/media/kilian/Extern_SSD/Robot_Vision/ZS6D/zs6d-sd-dino/sd-dino/third_party/ODISE/odise/model_zoo/configs/Panoptic/odise_label_coco_50e.py
test
No module 'xformers'. Proceeding without it.
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


KeyboardInterrupt: 

In [3]:

from PIL import Image
import pose_utils.img_utils as img_utils
import pose_utils.vis_utils as vis_utils
import numpy as np
import time
import matplotlib.pyplot as plt

# Loading a ground truth file to access segmentation masks to test zs6d:
with open(os.path.join(config['gt_path']), 'r') as f:
    data_gt = json.load(f)

img_id = '000048_1'
# img_id = '8'



for i in range(len(data_gt[img_id])):
    obj_number = i
    obj_id = data_gt[img_id][obj_number]['obj_id']
    cam_K = np.array(data_gt[img_id][obj_number]['cam_K']).reshape((3,3))
    bbox = data_gt[img_id][obj_number]['bbox_visib']

    img_path = "./test/000001.png"
    img = Image.open(img_path)

    mask = data_gt[img_id][obj_number]['mask_sam']
    mask = img_utils.rle_to_mask(mask)
    mask = mask.astype(np.uint8)

    start_time = time.time()
    
    # To estimate the objects Rotation R and translation t the input image, the object_id, a segmentation mask and camera matrix are necessary
    R_est, t_est = pose_estimator.get_pose(model, aug, img, str(obj_id), mask, cam_K, bbox=None)

    end_time = time.time()
    
    out_img = vis_utils.draw_3D_bbox_on_image(np.array(img), R_est, t_est, cam_K, data_gt[img_id][obj_number]['model_info'], factor=1.0)

    plt.imshow(out_img)
    plt.show()
    print(f"Pose estimation time: {end_time-start_time}")
    print(f"R_est: {R_est}")
    print(f"t_est: {t_est}")

Using cache found in /home/kilian/.cache/torch/hub/facebookresearch_dinov2_main


Shape of img1_desc (SD) features: torch.Size([1, 1, 3600, 384])
Shape of img1_batch: torch.Size([1, 3, 840, 840])


2024-07-03 20:30:59,431 - ERROR - Error in get_pose: shapes (4147200,) and (1161984,) not aligned: 4147200 (dim 0) != 1161984 (dim 0)


Shape of img1_desc_dino: torch.Size([1, 1, 3600, 768])
Shape of img1_desc (SD) normalized: torch.Size([1, 1, 3600, 384])
Shape of img1_desc_dino normalized: torch.Size([1, 1, 3600, 768])
Shape of img1_desc (Fused) after fusion: torch.Size([1, 1, 3600, 1152])
Shape of fused desc_sd_dino before squeezing: torch.Size([1, 1, 3600, 1152])
Shape of fused desc_sd_dino input before find_template_cpu: torch.Size([3600, 1152])
Shape of templates_desc[1] before find_template_cpu: torch.Size([43, 3026, 384])
Shape of desc_input: torch.Size([3600, 1152])
Shape of desc_templates: torch.Size([43, 3026, 384])
Shape of desc_input_flat: torch.Size([4147200])
Shape of first template_flat: torch.Size([1161984])


ValueError: shapes (4147200,) and (1161984,) not aligned: 4147200 (dim 0) != 1161984 (dim 0)